# Ray TuneとMLflowトラッキング: Pytorch Lightningトレーニングの最適化

このノートブックでは、Ray TuneとMLflowトラッキングの新たなインテグレーションを活用することで、どのようにPytorchのLightningモデルをトラッキングし、最適化するのかを、ステップバイステップで説明します。Ray Tuneは並列でのハイパーパラメーター探索に、MLflowは実験のロギングに活用します。さあ、始めましょう！

**参考資料**
- [RayとMLflow: 分散機械学習アプリケーションの本格運用 \- Qiita](https://qiita.com/taka_yayoi/items/078a5a0a74b18acdb03b)
- [How the Integrations Between Ray & MLflow Aids Distributed ML Production \- The Databricks Blog](https://databricks.com/blog/2021/02/03/ray-mlflow-taking-distributed-machine-learning-applications-to-production.html)

<table>
  <tr><th>作者</th><th>Amog Kamsetty / Databricks Japan</th></tr>
  <tr><td>日付</td><td>2021/05/23</td></tr>
  <tr><td>バージョン</td><td>1.0</td></tr>
  <tr><td>クラスター</td><td>DBR8.0ML</td></tr>
</table>

<img src="https://databricks.com/wp-content/themes/databricks/assets/images/header_logo_2x.png" alt="logo" width="150"/>

## ライブラリのインストール

まず初めに必要なライブラリ、MLflow(MLランタイムの場合はインストール済みです)、RayとPytorch Lightningをインストールします。

Rayのインストールにおいては、インテグレーションを活用するために最新のwheelをインストールする必要があります。しかし、Ray 1.2リリース以降は安定版をインストールするだけで大丈夫です。

以下のリンク先から使用しているOS、Pythonのバージョンに合致するものを選択する必要があります。
[Installing Ray — Ray v2\.0\.0\.dev0](https://docs.ray.io/en/master/installation.html)

In [0]:
%pip install pytorch-lightning==1.1.8
%pip install pytorch-lightning-bolts
%pip install torchvision
#%pip install mlflow

# Python v3.8、Ubuntu OS向けのRayの最新Wheelsをインストールします
# Ray 1.2リリース後は以下の行のインストールは不要となります
%pip install -U https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-2.0.0.dev0-cp38-cp38-manylinux2014_x86_64.whl

# Rayおよび、その他のRay Tuneの依存ライブラリをインストールします 
%pip install ray[tune]

Python interpreter will be restarted.
Collecting pytorch-lightning==1.1.8
 Using cached pytorch_lightning-1.1.8-py3-none-any.whl (696 kB)
Requirement already satisfied: future>=0.17.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from pytorch-lightning==1.1.8) (0.18.2)
Requirement already satisfied: numpy>=1.16.6 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from pytorch-lightning==1.1.8) (1.19.2)
Requirement already satisfied: torch>=1.3 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from pytorch-lightning==1.1.8) (1.7.1)
Collecting fsspec[http]>=0.8.1
 Using cached fsspec-2021.5.0-py3-none-any.whl (111 kB)
Requirement already satisfied: tensorboard>=2.2.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from pytorch-lightning==1.1.8) (2.4.1)
Requirement already satisfied: tqdm>=4.41.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from pytorch-lightning==1.1.8) (4.50.2)
Processing /root/.cache/pip/wheels/13/90/db/290ab3a34f2ef0b5a0f89235dc2d40fea83e77de84ed2dc05c/PyYAML-5.3.1-cp38-cp38-linux_x86_64.whl
Requirement already satisfied: typing_extensions in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from torch>=1.3->pytorch-lightning==1.1.8) (3.7.4.3)
Requirement already satisfied: aiohttp; extra == "http" in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from fsspec[http]>=0.8.1->pytorch-lightning==1.1.8) (3.6.3)
Requirement already satisfied: requests; extra == "http" in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from fsspec[http]>=0.8.1->pytorch-lightning==1.1.8) (2.24.0)
Requirement already satisfied: markdown>=2.6.8 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from tensorboard>=2.2.0->pytorch-lightning==1.1.8) (3.3.2)
Requirement already satisfied: six>=1.10.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from tensorboard>=2.2.0->pytorch-lightning==1.1.8) (1.15.0)
Requirement already satisfied: wheel>=0.26; python_version >= "3" in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from tensorboard>=2.2.0->pytorch-lightning==1.1.8) (0.35.1)
Requirement already satisfied: google-auth<2,>=1.6.3 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from tensorboard>=2.2.0->pytorch-lightning==1.1.8) (1.22.1)
Requirement already satisfied: google-auth-oauthlib<0.5,>=0.4.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from tensorboard>=2.2.0->pytorch-lightning==1.1.8) (0.4.2)
Requirement already satisfied: protobuf>=3.6.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from tensorboard>=2.2.0->pytorch-lightning==1.1.8) (3.13.0)
Requirement already satisfied: absl-py>=0.4 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from tensorboard>=2.2.0->pytorch-lightning==1.1.8) (0.11.0)
Requirement already satisfied: werkzeug>=0.11.15 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from tensorboard>=2.2.0->pytorch-lightning==1.1.8) (1.0.1)
Requirement already satisfied: tensorboard-plugin-wit>=1.6.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages (from tensorboard>=2.2.0-

## Pytorch Lightningモデルの定義

次に使用するPytorch Lightningモデルを定義します。この場合、Pytorch Lightningチュートリアルで使用されているMNISTデータセットに対するシンプルなモデルを使用します。

In [0]:
import torch
from torch.nn import functional as F
import pytorch_lightning as pl

class LightningMNISTClassifier(pl.LightningModule):
    def __init__(self, config, data_dir=None):
        super(LightningMNISTClassifier, self).__init__()

        self.data_dir = data_dir or os.getcwd()
        self.lr = config["lr"]
        layer_1, layer_2 = config["layer_1"], config["layer_2"]

        # mnist画像は(1, 28, 28)の形状です (channels, width, height)
        self.layer_1 = torch.nn.Linear(28 * 28, layer_1)
        self.layer_2 = torch.nn.Linear(layer_1, layer_2)
        self.layer_3 = torch.nn.Linear(layer_2, 10)
        self.accuracy = pl.metrics.Accuracy()

    def forward(self, x):
        batch_size, channels, width, height = x.size()
        x = x.view(batch_size, -1)
        x = self.layer_1(x)
        x = torch.relu(x)
        x = self.layer_2(x)
        x = torch.relu(x)
        x = self.layer_3(x)
        x = torch.log_softmax(x, dim=1)
        return x

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        acc = self.accuracy(logits, y)
        self.log("ptl/train_loss", loss)
        self.log("ptl/train_accuracy", acc)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        acc = self.accuracy(logits, y)
        return {"val_loss": loss, "val_accuracy": acc}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        avg_acc = torch.stack([x["val_accuracy"] for x in outputs]).mean()
        self.log("ptl/val_loss", avg_loss)
        self.log("ptl/val_accuracy", avg_acc)

## トレーニング関数の作成、モデルのロギング

モデルをトレーニングするための関数を定義します。

ここで大切なのは、トレーニング関数に `mlflow_mixin` デコレータを追加することです。デコレータを追加することで、トレーニング関数で、あらゆるmlflow.trackingメソッドを呼び出すことができ、適切なMLflowランが自動的に記録されます。

この例では、トレーニングの開始前にシンプルに `mlflow.pytorch.autolog()` を呼び出します。これにより、明示的に記録を指示することなしに、Pytorch Lightningモデルの全てのメトリクス、パラメーター、モデルアーティファクトを自動的にロギングします。

そして、Ray TuneとPytorch Lightningの連携によって、 `TuneReportCallback` がモデルのトレーニングが行われた際に中間結果をTuneにレポートします。

最後に必要なことは、適切なDatabricksの認証情報を設定し、リモートのプロセスからDatabricksサーバーにログインできるようにするということです。

In [0]:
from pl_bolts.datamodules.mnist_datamodule import MNISTDataModule
from ray.tune.integration.mlflow import mlflow_mixin
from ray.tune.integration.pytorch_lightning import TuneReportCallback

@mlflow_mixin
def train_mnist_tune(config):
    host = config["mlflow"]["tracking_uri"]
    token = config["mlflow"]["token"]
    if "databricks" in host:
      import os
      os.environ["DATABRICKS_HOST"] = host
      os.environ["DATABRICKS_TOKEN"] = token
    model = LightningMNISTClassifier(config, config["data_dir"])
    dm = MNISTDataModule(
        data_dir=data_dir, num_workers=0, batch_size=config["batch_size"])
    metrics = {"val_loss": "ptl/val_loss", "val_acc": "ptl/val_accuracy"}
    mlflow.pytorch.autolog()
    trainer = pl.Trainer(
        max_epochs=config["num_epochs"],
        gpus=config["num_gpus"],
        progress_bar_refresh_rate=0,
        callbacks=[TuneReportCallback(metrics, on="validation_end")])
    trainer.fit(model, dm)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages/pl_bolts/utils/warnings.py:30: UserWarning: You want to use `wandb` which is not installed yet, install it with `pip install wandb`.
 stdout_func(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages/pl_bolts/utils/warnings.py:30: UserWarning: You want to use `gym` which is not installed yet, install it with `pip install gym`.
 stdout_func(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
 warnings.warn(

## ハイパーパラメーター探索の実行

Ray Tuneを使うために最後に必要なことは、我々のモデルに対する並列ハイパーパラメーター探索処理を起動することです。

ここでは、使用するハイパーパラメーターの探索空間をシンプルに定義します。そして、ハイパーパラメーター設定に基づいて、Ray Tuneが多くのトレーニングを並列実行することで、マシンやクラスターの全てのリソースを最大限に活用します。トレーニング関数を定義する際に `mlflow_mixin` デコレータを使用したので、それぞれのトレーニングの実行(ラン)はMLflowに記録されます。

ここでは、MLflowのトラッキングURI、あるいは、実験ログや結果を格納する場所を指定する必要があります。これは、ローカルのディレクトリ、サーバーのURI、HTTPサーバーやDatabricksのワークスペースでも構いません。

そして最後に、 `mixin` はリモートのRayプロセスからMLflowにロギングするので、適切なDatabricksの認証情報がプロセスに引き渡される必要があります。Databricks上のMLflowを使っていない場合には、このステップは不要です。

### データのダウンロード

In [0]:
data_dir = "./mnist"
# データのダウンロード
MNISTDataModule(data_dir=data_dir).prepare_data()

Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw/train-images-idx3-ubyte.gz
0it [00:00, ?it/s] 0%| | 0/9912422 [00:00<?, ?it/s] 0%| | 24576/9912422 [00:00<01:03, 154871.73it/s] 0%| | 40960/9912422 [00:00<01:12, 137042.23it/s] 1%| | 57344/9912422 [00:00<01:18, 125086.37it/s] 1%| | 73728/9912422 [00:00<01:24, 116974.97it/s] 1%| | 90112/9912422 [00:01<01:27, 111905.47it/s] 1%| | 106496/9912422 [00:01<01:30, 108947.43it/s] 1%| | 122880/9912422 [00:01<01:32, 106289.74it/s] 1%|▏ | 139264/9912422 [00:01<01:32, 105333.22it/s] 2%|▏ | 155648/9912422 [00:01<01:33, 104652.36it/s] 2%|▏ | 172032/9912422 [00:01<01:35, 101721.76it/s] 2%|▏ | 188416/9912422 [00:01<01:33, 103913.86it/s] 2%|▏ | 204800/9912422 [00:02<01:34, 103240.24it/s] 2%|▏ | 221184/9912422 [00:02<01:34, 103097.77it/s] 2%|▏ | 237568/9912422 [00:02<01:34, 102682.28it/s] 3%|▎ | 253952/9912422 [00:02<01:34, 102228.09it/s] 3%|▎ | 270336/9912422 [00:02<01:34, 102420.67it/s] 3%|▎ | 286720/9912422 [00:02<01:35, 101259.54it/s] 3%|▎ | 303104/9912422 [00:03<01:33, 102629.88it/s] 3%|▎ | 319488/9912422 [00:03<01:33, 102455.61it/s] 3%|▎ | 335872/9912422 [00:03<01:34, 101286.53it/s] 4%|▎ | 352256/9912422 [00:03<01:34, 101479.66it/s] 4%|▎ | 368640/9912422 [00:03<01:34, 101325.37it/s] 4%|▍ | 385024/9912422 [00:03<01:33, 102132.65it/s] 4%|▍ | 401408/9912422 [00:04<01:32, 103056.07it/s] 4%|▍ | 417792/9912422 [00:04<01:32, 102761.59it/s] 4%|▍ | 434176/9912422 [00:04<01:32, 102562.46it/s] 5%|▍ | 450560/9912422 [00:04<01:33, 101727.87it/s] 5%|▍ | 466944/9912422 [00:04<01:32, 102490.57it/s] 5%|▍ | 483328/9912422 [00:04<01:32, 102360.66it/s] 5%|▌ | 499712/9912422 [00:05<01:32, 102272.03it/s] 5%|▌ | 516096/9912422 [00:05<01:32, 102007.07it/s] 5%|▌ | 532480/9912422 [00:05<01:31, 102028.64it/s] 6%|▌ | 548864/9912422 [00:05<01:31, 102179.52it/s] 6%|▌ | 565248/9912422 [00:05<01:34, 98720.83it/s]  6%|▌ | 581632/9912422 [00:05<01:30, 102871.46it/s] 6%|▌ | 598016/9912422 [00:05<01:30, 102678.39it/s] 6%|▌ | 614400/9912422 [00:06<01:39, 93912.17it/s]  6%|▋ | 630784/9912422 [00:06<01:28, 104856.80it/s] 7%|▋ | 647168/9912422 [00:06<01:28, 104256.57it/s] 7%|▋ | 663552/9912422 [00:06<01:29, 103033.28it/s] 7%|▋ | 679936/9912422 [00:06<01:28, 103882.15it/s] 7%|▋ | 696320/9912422 [00:06<01:23, 109956.71it/s] 7%|▋ | 712704/9912422 [00:07<01:20, 114160.71it/s] 7%|▋ | 729088/9912422 [00:07<01:17, 118052.25it/s] 8%|▊ | 745472/9912422 [00:07<01:15, 120988.71it/s] 8%|▊ | 761856/9912422 [00:07<01:14, 122958.06it/s] 8%|▊ | 778240/9912422 [00:07<01:13, 124167.14it/s] 8%|▊ | 794624/9912422 [00:07<01:12, 125042.75it/s] 8%|▊ | 811008/9912422 [00:07<01:12, 125603.60it/s] 8%|▊ | 827392/9912422 [00:07<01:11, 126600.03it/s] 9%|▊ | 843776/9912422 [00:08<01:11, 126312.33it/s] 9%|▊ | 860160/9912422 [00:08<01:11, 127320.02it/s] 9%|▉ | 876544/9912422 [00:08<01:12, 125402.40it/s] 9%|▉ | 892928/9912422 [00:08<01:10, 127532.67it/s] 9%|▉ | 909312/9912422 [00:08<01:10, 127184.58it/s] 9%|▉ | 925696/9912422 [00:08<01:10, 127809.28it/s] 10%|▉ | 942080/9912422 [00:08<01:10, 127621.51it/s] 10%|▉ | 958464/9912422 [00:08<01:09, 127933.35it/s] 10%|▉ | 974848/9912422 [00:09<01:10, 127552.20it/s] 10%|▉ | 991232/9912422 [00:09<01:09, 127750.22it/s] 10%|█ | 1007616/9912422 [00:09<01:10, 127045.20it/s] 10%|█ | 1024000/9912422 [00:09<01:09, 127784.10it/s] 10%|█ | 1040384/9912422 [00:09<01:09, 127797.86it/s] 11%|█ | 1056768/9912422 [00:09<01:09, 127233.89it/s] 11%|█ | 1073152/9912422 [00:09<01:09, 126998.18it/s] 11%|█ | 1089536/9912422 [00:10<01:09, 127371.25it/s] 11%|█ | 1105920/9912422 [00:10<01:09, 125997.30it/s] 11%|█▏ | 1122304/9912422 [00:10<01:08, 127394.31it/s] 11%|█▏ | 1138688/9912422 [00:10<01:08, 128366.90it/s] 12%|█▏ | 1155072/9912422 [00:10<01:08, 128040.38it/s] 12%|█▏ | 1171456/9912422 [00:10<01:08, 127792.70it/s] 12%|█▏ | 1187840/9912422 [00:10<01:08, 127891.79it/s] 12%|█▏ | 1204224/9912422 [00:10<01:10, 122924.47it/s] 12%|█

### MLflowの設定

In [0]:
import mlflow

# 既存のエクスペリメント名、存在しない場合には新たにDatabricksワークスペースに作成したエクスペリメント名を指定します
experiment_name = "/Users/takaaki.yayoi@databricks.com/20210521_Ray_MLflow/Ray integration"

mlflow.set_experiment(experiment_name)

8192it [00:00, 18397.06it/s]

In [0]:
# Databricks認証情報の取得
import os
from mlflow.tracking._tracking_service import utils
# 注意: リモートトラッキングサーバーにロギングしていない場合、例えば、ローカルファイルシステムにロギングしている場合には、 get_host_creds は undefined となります
host_creds = utils._get_store().get_host_creds()
token = host_creds.token
host = host_creds.host

### ハイパーパラメーターの定義

In [0]:
from ray import tune

# 1 GPUを使用してトレーニングを行う場合には1を指定してください
num_gpus_per_run = 0

# ハイパーパラメーター探索空間の指定
# Tuneはこの探索空間を分解して、トレーニング関数に設定を引き渡します
config = {
        "layer_1": tune.choice([32, 64, 128]),
        "layer_2": tune.choice([64, 128, 256]),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([32, 64, 128]),
        # MLflowの設定を引き渡します。Databricksで実行していない場合にはトークンは不要です
        "mlflow": {
            "experiment_name": experiment_name,
            "tracking_uri": host,
            "token": token
        },
        "data_dir": "./mnist",
        "num_epochs": 2,
        "num_gpus": num_gpus_per_run
    }

### tune.runの呼び出し

全ての準備が整ったので、 `tune.run` を呼び出します。出力結果から、Tuneが数多くの異なるトレーニングランを並列で実行していることが分かると思います。これによって、容易に並列ハイパーパラメーターチューニングを実行できます。また、 `resources_per_trial` フィールドを指定することでGPUを活用することもできます。

In [0]:
# DatabricksでRayを動作させるHackについては以下を参照ください
# https://forums.databricks.com/questions/45772/is-there-step-by-step-guide-how-to-setup-ray-clust.html
import sys

sys.stdout.fileno = lambda: False

In [0]:
import ray

In [0]:
analysis = tune.run(
        train_mnist_tune,
        resources_per_trial={
            "cpu": 1,
            "gpu": num_gpus_per_run
        },
        metric="val_loss",
        mode="min",
        config=config,
        # ハイパーパラメーター探索空間からいくつの異なるサンプルを試すか?
        num_samples=2,
        name="tune_mnist")

print("探索から得られたベストなハイパーパラメーター: ", analysis.best_config)

2021-05-23 00:39:18,517	INFO services.py:1272 -- View the Ray dashboard at http://127.0.0.1:8265 
9920512it [01:20, 213041.18it/s] 2021-05-23 00:39:21,690	WARNING function_runner.py:544 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
1654784it [00:12, 133372.12it/s] 
<IPython.core.display.HTML object>
 (pid=3452) /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages/pl_bolts/utils/warnings.py:30: UserWarning: You want to use `wandb` which is not installed yet, install it with `pip install wandb`.
 (pid=3452) stdout_func(
 (pid=3449) /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages/pl_bolts/utils/warnings.py:30: UserWarning: You want to use `wandb` which is not installed yet, install it with `pip install wandb`.
 (pid=3449) stdout_func(
 (pid=3452) /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages/pl_bolts/utils/warnings.py:30: UserWarning: You want to use `gym` which is not installed yet, install it with `pip install gym`.
 (pid=3452) stdout_func(
 (pid=3449) /local_disk0/.ephemeral_nfs/envs/pythonEnv-e424334a-e132-4563-9eda-f50b2740fc46/lib/python3.8/site-packages/pl_bolts/utils/warnings.py:30: UserWarning: You want to use `gym` which is not installed yet, install it with `pip install gym`.
 (pid=3449) stdout_func(
 (pid=3452) GPU available: False, used: False
 (pid=3452) TPU available: None, using: 0 TPU cores
 (pid=3452) 2021/05/23 00:39:25 INFO mlflow.utils.autologging_utils: pytorch autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pytorch workflow to the MLflow run with ID 'd413d54396264deba684d01f2299ced4'
 (pid=3452) 0it [00:00, ?it/s]
 (pid=3452) Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw/train-images-idx3-ubyte.gz
 (pid=3449) GPU available: False, used: False
 (pid=3449) TPU available: None, using: 0 TPU cores
 (pid=3449) 2021/05/23 00:39:25 INFO mlflow.utils.autologging_utils: pytorch autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pytorch workflow to the MLflow run with ID '8a18aa59df9a4043bb1858fbe0370d79'
 (pid=3449) 0it [00:00, ?it/s]
 (pid=3449) Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw/train-images-idx3-ubyte.gz
 (pid=3452)  0%| | 0/9912422 [00:00<?, ?it/s]
 (pid=3452)  0%| | 32768/9912422 [00:00<00:31, 316634.38it/s]
 (pid=3449)  0%| | 0/9912422 [00:00<?, ?it/s]
 (pid=3452)  1%| | 57344/9912422 [00:00<00:34, 285106.97it/s]
 (pid=3449)  0%| | 32768/9912422 [00:00<00:42, 230126.57it/s]
 (pid=3452)  1%| | 81920/9912422 [00:00<00:41, 236762.42it/s]
 (pid=3452)  1%| | 98304/9912422 [00:00<00:48, 200928.51it/s]
 (pid=3449)  1%| | 57344/9912422 [00:00<00:55, 176937.85it/s]
 (pid=3452)  1%| | 122880/9912422 [00:00<00:49, 197030.23it/s]
 (pid=3449)  1%| | 90112/9912422 [00:00<00:48, 200675.79it/s]
 (pid=3449)  1%| | 114688/9912422 [00:00<00:51, 189930.29it/s]
 (pid=3452)  1%|▏ | 147456/9912422 [00:00<00:51, 187980.83it/s]
 (pid=3452)  2%|▏ | 172032/9912422 [00:01<00:53, 181304.82it/s]
 (pid=3449)  1%|▏ | 139264/9912422 [00:00<00:53, 183762.38it/s]
 (pid=3452)  2%|▏ | 196608/9912422 [00:01<00:54, 178487.93it/s]
 (pid=3449)  2%|▏ | 163840/9912422 [00:01<00:54, 177922.77it/s]
 (pid=3452)  2%|▏ | 221184/9912422 [00:01<00:55, 175972.60it/s]
 (pid=3449)  2%|▏ | 188416/9912422 [00:01<00:55, 174013.26it/s]
 (pid=3452)  2%|▏ | 245760/9912422 [00:01<00:55, 172895.17it/s]
 (pid=3449)  2%|▏ | 212992/9912422 [00:01<00:55, 174427.49it/s]
 (pid=3452)  3%|▎ | 270336/9912422 [00:01<00:55, 172967.92it/s]
 (pid=3449)  2%|▏ | 237568/991242

### MLflowを使ったランの可視化、クエリー

実験の処理が終了した後は、MLflowを使って、UIでランを可視化したり、プログラムで問い合わせを行うことができます。

In [0]:
from mlflow.tracking import MlflowClient
client = MlflowClient()
runs = client.search_runs(
            [mlflow.get_experiment_by_name(experiment_name).experiment_id])

In [0]:
# ベストなランを検索し、ベストランのメトリクスを最終結果として記録します
best_val_loss = float("inf")
for r in runs:
    if "best_run" not in r.data.tags and r.data.metrics["ptl/val_loss"] < best_val_loss:
        best_run = r
        best_train_loss = r.data.metrics["ptl/train_loss"]
        best_val_acc = r.data.metrics["ptl/val_accuracy"]
        best_train_acc = r.data.metrics["ptl/train_accuracy"]

In [0]:
mlflow.end_run()
with mlflow.start_run():
  mlflow.set_tag("best_run", best_run.info.run_id)
  mlflow.log_metrics(
      {
          "train_loss": best_train_loss,
          "val_loss": best_val_loss,
          "train_accuracy": best_train_acc,
          "val_accuracy": best_val_acc
      }
  )

# END